# 🎭 ROOP Batch Processor - Google Colab

**Instrucciones:**
1. Ejecuta CELDA 1 (se reiniciará automáticamente)
2. Después del reinicio, ejecuta CELDA 2
3. Sube tus archivos en CELDA 3
4. Procesa en CELDA 4
5. Descarga en CELDA 5

In [ ]:
# ==========================================
# 📦 CELDA 1: Setup inicial (SE REINICIARÁ)
# ==========================================

!git clone https://github.com/CDavidDv/RoopConClaude.git 2>/dev/null || echo "Ya clonado"
%cd /content/RoopConClaude

# Instalar dependencias del sistema
!apt-get update -qq 2>/dev/null
!apt-get install -y -qq libcufft10 ffmpeg 2>/dev/null

# Actualizar pip y setuptools primero (evita errores de build)
!pip install -q --upgrade pip setuptools wheel cython

# Instalar numpy correcto PRIMERO
!pip install -q numpy==1.23.5

print("\n" + "="*50)
print("⚠️  REINICIANDO RUNTIME...")
print("="*50)
print("Después del reinicio, ejecuta la CELDA 2")

# Reiniciar para aplicar numpy
import os
os.kill(os.getpid(), 9)

In [ ]:
# ==========================================
# 📦 CELDA 2: Instalar dependencias
# ==========================================

%cd /content/RoopConClaude

# Verificar numpy
import numpy as np
print(f"NumPy: {np.__version__}")

if not np.__version__.startswith("1."):
    print("❌ NumPy incorrecto. Ejecuta CELDA 1 de nuevo.")
else:
    print("✅ NumPy correcto")
    print("\n📦 Instalando dependencias...")
    
    # Actualizar herramientas de build
    !pip install -q --upgrade pip setuptools wheel cython
    
    # Instalar en orden correcto
    !pip install -q onnx==1.14.0
    !pip install -q onnxruntime-gpu==1.18.0
    !pip install -q tensorflow==2.13.0
    !pip install -q opencv-python==4.8.0.74
    !pip install -q pillow==10.0.0
    !pip install -q protobuf==4.23.4
    !pip install -q psutil==5.9.5
    !pip install -q tqdm==4.65.0
    
    # InsightFace necesita estas dependencias
    !pip install -q albumentations
    !pip install -q insightface==0.7.3
    !pip install -q gfpgan==1.3.8
    
    # Verificar instalación
    print("\n" + "="*50)
    print("🎯 VERIFICACIÓN")
    print("="*50)
    
    import onnxruntime as ort
    import torch
    
    print(f"📦 ONNX Runtime: {ort.__version__}")
    print(f"🎯 Providers: {ort.get_available_providers()}")
    print(f"🔥 CUDA: {torch.cuda.is_available()}")
    if torch.cuda.is_available():
        print(f"🖥️  GPU: {torch.cuda.get_device_name(0)}")
    print("="*50)
    print("\n✅ Instalación completada! Continúa con CELDA 3")

In [ ]:
# ==========================================
# 📤 CELDA 3: Subir archivos
# ==========================================

from google.colab import files
import os

%cd /content/RoopConClaude

# Crear carpetas
!mkdir -p source inputVideos outputVideos

# Subir imagen fuente
print("📸 Sube tu imagen de referencia (JPG/PNG):")
uploaded = files.upload()
for filename in uploaded:
    os.rename(filename, f"source/{filename}")
    print(f"✅ Guardado: source/{filename}")

# Subir videos
print("\n🎬 Sube tus videos (puedes seleccionar múltiples):")
uploaded = files.upload()
for filename in uploaded:
    os.rename(filename, f"inputVideos/{filename}")
    print(f"✅ Guardado: inputVideos/{filename}")

# Mostrar resumen
print("\n" + "="*50)
print("📁 ARCHIVOS SUBIDOS")
print("="*50)
!ls -la source/
print()
!ls -la inputVideos/ | head -20
print("\n✅ Archivos listos! Continúa con CELDA 4")

In [ ]:
# ==========================================
# 🚀 CELDA 4: Procesar videos
# ==========================================

%cd /content/RoopConClaude

# Keep-alive para evitar timeout
from IPython.display import display, Javascript
display(Javascript('''
function ClickConnect(){
  console.log("Keeping session alive...");
  document.querySelector("colab-toolbar-button#connect").click()
}
setInterval(ClickConnect, 60000)
'''))

# ============================================
# OPCIONES DE PROCESAMIENTO (descomenta UNA):
# ============================================

# Opción A: Procesar TODOS los videos
!python runbatch.py

# Opción B: Procesar un RANGO (recomendado para evitar timeout)
# !python runbatch.py 1-40

# Opción C: Procesar UN SOLO video para probar
# !python runbatch.py 1

In [ ]:
# ==========================================
# 📥 CELDA 5: Descargar resultados
# ==========================================

from google.colab import files
from pathlib import Path

%cd /content/RoopConClaude

# Listar videos procesados
output_files = sorted(Path('outputVideos').glob('*.mp4'))

if not output_files:
    print("❌ No hay videos procesados aún")
else:
    print(f"📥 Videos procesados: {len(output_files)}")
    print("="*50)
    
    total_size = 0
    for i, video in enumerate(output_files, 1):
        size_mb = video.stat().st_size / 1e6
        total_size += size_mb
        print(f"  {i}. {video.name} ({size_mb:.1f} MB)")
    
    print("="*50)
    print(f"💾 Total: {total_size:.1f} MB")
    print("\n⬇️ Descargando...")
    
    for video in output_files:
        files.download(str(video))
    
    print("\n✅ Descarga completada!")

In [ ]:
# ==========================================
# 🔄 CELDA EXTRA: Actualizar código
# ==========================================
# Usa esta celda si hay actualizaciones en el repositorio

%cd /content/RoopConClaude
!git pull
print("✅ Código actualizado!")

In [ ]:
# ==========================================
# 📤 CELDA EXTRA: Subir más videos
# ==========================================
# Usa esta celda para agregar más videos sin reiniciar

from google.colab import files
import os

%cd /content/RoopConClaude

print("🎬 Sube más videos:")
uploaded = files.upload()
for filename in uploaded:
    os.rename(filename, f"inputVideos/{filename}")
    print(f"✅ Guardado: inputVideos/{filename}")

# Contar videos
!echo "📊 Total videos en inputVideos:" && ls inputVideos/*.mp4 2>/dev/null | wc -l